In [ ]:
# our goal here is to take a bunch of historical data on users blockchain activity
# we then want to use this to predict if the user will stay in an LP position or remove their LP position

In [1]:
import pandas as pd 

optiData = pd.read_csv("optiTrans.csv")
avaxData = pd.read_csv("avaxTrans.csv")
arbiData = pd.read_csv("arbiTrans.csv")
polyData = pd.read_csv("polyTrans.csv")

"""
Historical transaction data aggregrated from different chains via flipside 
"""

df = pd.concat([optiData, avaxData, arbiData, polyData])
df['FIRST_TRANSACTION'] =  df['FIRST_TRANSACTION'].astype('datetime64[ns]')
df['LAST_TRANSACTION'] =  df['LAST_TRANSACTION'].astype('datetime64[ns]')


veloDf = pd.read_csv("lpInfo.csv")
veloDf['Time'] =  veloDf['BLOCK_TIMESTAMP'].astype('datetime64[ns]')


In [2]:
df.head()

,FIRST_TRANSACTION,LAST_TRANSACTION,TOTAL,N_TRANSACTIONS,FROM_ADDRESS,TO_ADDRESS,CONTRACT_ADDRESS,CHAIN,SYMBOL
0,2022-09-02 07:23:58,2022-10-17 13:47:16,73421879781634400000,741,0x85149247691df622eaf1a8bd0cafd40bc45154a9,0xff40b156a428758e2d37d95bbc3d1e185a394a66,0x4200000000000000000000000000000000000006,OPTI,WETH
1,2022-09-07 09:56:51,2022-09-30 14:11:08,1900009424984,2,0x0172e05392aba65366c4dbbb70d958bbf43304e4,0x0000000000000000000000000000000000000000,0xfb00ac187a8eb5afae4eace434f493eb62672df7,OPTI,variableDebtOptUSDT
2,2022-09-05 08:05:45,2022-09-08 05:35:11,1003058528997,5,0x0000000000000000000000000000000000000000,0x31754e3c96cad2459f66b47f158c4399e8d4f79a,0x625e7708f30ca75bfd92586e17077590c60eb4cd,OPTI,aOptUSDC
3,2021-11-24 07:11:58,2021-12-25 23:09:34,9.07651239690097e+22,3,0xbbb33d2e7bd7ddc722e53da9ca8ee97df41cfabf,0xf181ed90d6cfac84b8073fdea6d34aa744b41810,0x56900d66d74cb14e3c86895789901c9135c95b16,OPTI,hDAI
4,2021-12-26 06:26:30,2021-12-26 06:26:30,1.50083615885431e+22,1,0x0000000000000000000000000000000000000000,0x48a5a6a01ba89cddf97d2d552923d5a11401ed19,0x06bad08305074da59fe98d0e85bad8f524e167df,OPTI,aeD-0x41503100000000000000000000


In [3]:
veloDf.head()

,BLOCK_NUMBER,BLOCK_TIMESTAMP,TX_HASH,EVENT_INDEX,ORIGIN_FUNCTION_SIGNATURE,ORIGIN_FROM_ADDRESS,ORIGIN_TO_ADDRESS,POOL_ADDRESS,POOL_NAME,POOL_TYPE,...,TOKEN0_AMOUNT,TOKEN1_AMOUNT,TOKEN0_AMOUNT_USD,TOKEN1_AMOUNT_USD,TOKEN0_ADDRESS,TOKEN1_ADDRESS,LP_TOKEN_ACTION,LP_TOKEN_AMOUNT,LP_TOKEN_AMOUNT_USD,Time
0,72291734,2023-02-04 18:06:29.000,0x4d1ae6db8d6270c7de84a626784dd6bc91635728a947...,6,0x7ae7cfb5,0xc8d71390f466aecef721dcab073b6190f0ec9639,0x1ec50880101022c11530a069690f5446d1464592,0xd16232ad60188b68076a235c65d692090caba155,StableV1 AMM - USDC/sUSD,stable,...,5312.423521,4680.172772,5312.42,4680.17,0x7f5c764cbc14f9669b88837ca1490cca17c31607,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,mint,0.004996,9992.59,2023-02-04 18:06:29
1,72305318,2023-02-04 20:45:12.000,0x1d1a9a56fbf48d912d52a25935f6c5f484234ac4559a...,4,0x5a47ddc3,0xbbb33d2e7bd7ddc722e53da9ca8ee97df41cfabf,0x9c12939390052919af3155f41bf4160fd3666a6f,0xb343dae0e7fe28c16ec5dca64cb0c1ac5f4690ac,StableV1 AMM - wstETH/sETH,stable,...,35.132389,75.894382,65320.95,127829.67,0x1f32b1c2345538c0c6f582fcb022739c4a194ebb,0xe405de8f52ba7559f9df3c368500b6e6ae6cee49,mint,55.097142,193150.62,2023-02-04 20:45:12
2,72307312,2023-02-04 21:17:09.000,0x3b99010d3802e824defcb3e16e66ac25a4f73bcac3f9...,4,0x5a47ddc3,0xb755885151df63f2d7d3431c66f48c69b83850cb,0x9c12939390052919af3155f41bf4160fd3666a6f,0xb343dae0e7fe28c16ec5dca64cb0c1ac5f4690ac,StableV1 AMM - wstETH/sETH,stable,...,27.960187,60.400000,51886.56,101634.48,0x1f32b1c2345538c0c6f582fcb022739c4a194ebb,0xe405de8f52ba7559f9df3c368500b6e6ae6cee49,mint,43.848839,153521.04,2023-02-04 21:17:09
3,72304851,2023-02-04 20:37:56.000,0xf6174298c9a6afe2e1d58f9392ad5bb2a80d95eca603...,44,0xde5f6268,0x82bd1487843704d2fd846788b69e9a7e06060f8f,0x0f21607f78ee4bdd6895a7517851531b03c2c58a,0xffb4fd14e840abd3ebd6cd026332b2302948a3fc,StableV1 AMM - DAI/MAI,stable,...,2.420696,3.374272,2.42,NaN,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,0xdfa46478f9e5ea86d57387849598dbfb2e964b02,mint,2.894313,NaN,2023-02-04 20:37:56
4,72284884,2023-02-04 17:04:22.000,0x252c0b17449ed8dfa85d28c3e0b5e599085fb1ffa59b...,5,0x5a47ddc3,0x277d250383807fdb4840e7901a262f5c4c4be64b,0x9c12939390052919af3155f41bf4160fd3666a6f,0x794299137282e5d3af56616624ab918d1b2becf3,VolatileV1 AMM - wstETH/USDC,volatile,...,123.992948,229982.327549,229979.64,229982.33,0x1f32b1c2345538c0c6f582fcb022739c4a194ebb,0x7f5c764cbc14f9669b88837ca1490cca17c31607,mint,0.001020,459961.97,2023-02-04 17:04:22


In [4]:
symbols = df.SYMBOL.value_counts()
topAssets = symbols.index[0:15]

In [5]:
chains = df.CHAIN.unique()

In [6]:
date = pd.to_datetime("2023-1-1")

In [7]:
veloDfPred = veloDf[veloDf.Time < date]
veloDfPred['Users_Pool'] = veloDfPred.ORIGIN_FROM_ADDRESS + "SPLIT" + veloDfPred.POOL_NAME

users = veloDfPred.Users_Pool.unique()

userDf = pd.DataFrame({'Users_Pool' : list(users)})

/tmp/ipykernel_46420/1690657328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  veloDfPred['Users_Pool'] = veloDfPred.ORIGIN_FROM_ADDRESS + "SPLIT" + veloDfPred.POOL_NAME


In [8]:
veloDfPred.head()

,BLOCK_NUMBER,BLOCK_TIMESTAMP,TX_HASH,EVENT_INDEX,ORIGIN_FUNCTION_SIGNATURE,ORIGIN_FROM_ADDRESS,ORIGIN_TO_ADDRESS,POOL_ADDRESS,POOL_NAME,POOL_TYPE,...,TOKEN1_AMOUNT,TOKEN0_AMOUNT_USD,TOKEN1_AMOUNT_USD,TOKEN0_ADDRESS,TOKEN1_ADDRESS,LP_TOKEN_ACTION,LP_TOKEN_AMOUNT,LP_TOKEN_AMOUNT_USD,Time,Users_Pool
85,29738931,2022-10-16 17:05:35.000,0x1e7bf43d6dcc302bf01e36fac0d1ff9e9eac38be0a33...,52,0xf5d07b60,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29,0x9b50b06b81f033ca86d70f0a44f30bd7e0155737,0x0d693efd716021878d5979fab4cf8f6c1b7ce450,StableV1 AMM - sUSD/LUSD,stable,...,4.056210,6.56,4.14,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,mint,48173.517087,10.70,2022-10-16 17:05:35,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29SPLI...
86,29768354,2022-10-16 21:49:07.000,0x35498a854a22affca132e110f342bcf55ca646bcc595...,5,0x5a47ddc3,0x5c70b3f22595caf8314f77dabc628363aaee18ad,0x9c12939390052919af3155f41bf4160fd3666a6f,0xe8537b6ff1039cb9ed0b71713f697ddbadbb717d,VolatileV1 AMM - VELO/USDC,volatile,...,399.998475,399.24,399.78,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,0x7f5c764cbc14f9669b88837ca1490cca17c31607,mint,0.002500,799.02,2022-10-16 21:49:07,0x5c70b3f22595caf8314f77dabc628363aaee18adSPLI...
87,29752771,2022-10-16 19:17:26.000,0x6ee2d6dbc47e6a3b0ca3f459b7d99f6daa229194e2bd...,5,0x0dede6c4,0x8f360352100ddf1134b3bdcaa4792637bb5a4248,0x9c12939390052919af3155f41bf4160fd3666a6f,0xa4549b89a39f76d9d28415474aed7d06ec9935fe,StableV1 AMM - USDC/TUSD,stable,...,24914.482317,25080.88,NaN,0x7f5c764cbc14f9669b88837ca1490cca17c31607,0xcb59a0a753fdb7491d5f3d794316f1ade197b21e,burn,0.025000,NaN,2022-10-16 19:17:26,0x8f360352100ddf1134b3bdcaa4792637bb5a4248SPLI...
88,29738931,2022-10-16 17:05:35.000,0x1e7bf43d6dcc302bf01e36fac0d1ff9e9eac38be0a33...,12,0xf5d07b60,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29,0x9b50b06b81f033ca86d70f0a44f30bd7e0155737,0x0d693efd716021878d5979fab4cf8f6c1b7ce450,StableV1 AMM - sUSD/LUSD,stable,...,37050.505771,59959.43,37791.52,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,mint,48173.517087,97750.95,2022-10-16 17:05:35,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29SPLI...
89,29739124,2022-10-16 17:07:06.000,0xeabd81fa40bfa1e6a86905f70cbf58c15aaa7071c5b1...,14,0xf5d07b60,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29,0x9b50b06b81f033ca86d70f0a44f30bd7e0155737,0x0d693efd716021878d5979fab4cf8f6c1b7ce450,StableV1 AMM - sUSD/LUSD,stable,...,1068.743400,1665.85,1090.12,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,mint,0.155100,2755.97,2022-10-16 17:07:06,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29SPLI...


In [9]:
dfMints = veloDfPred[veloDfPred.LP_TOKEN_ACTION == 'mint']
dfBurns = veloDfPred[veloDfPred.LP_TOKEN_ACTION == 'burn']

def getTotalMints(row) : 
    return dfMints[dfMints.Users_Pool == row.Users_Pool]['LP_TOKEN_AMOUNT'].sum()
    
def getTotalBurns(row) : 
    return dfBurns[dfBurns.Users_Pool == row.Users_Pool]['LP_TOKEN_AMOUNT'].sum()


userDf['totalMints'] = userDf.apply(getTotalMints, axis = 1)
userDf['totalBurns'] = userDf.apply(getTotalBurns, axis = 1)

In [10]:
userDf['NetMints'] = userDf['totalMints'] - userDf['totalBurns']

In [11]:
userDf[userDf['NetMints'] > 0]

,Users_Pool,totalMints,totalBurns,NetMints
0,0x90ceeaffb3da888d32b02cf359f9e6a44ac8fa29SPLI...,96349.183968,49908.567778,4.644062e+04
3,0x5d368c382ae92fba52233b95c633c96fe49d0dc5SPLI...,0.085417,0.000000,8.541745e-02
5,0x5c70b3f22595caf8314f77dabc628363aaee18adSPLI...,112111.953212,112111.953212,1.455192e-11
6,0xed78afd8acba4b56703c4c0ac6cdfa6b7433c1aaSPLI...,612.174074,591.017734,2.115634e+01
7,0xc4e70f1c8a7d74afbb5da56d4b3e307c125fefd7SPLI...,0.057269,0.018145,3.912416e-02
...,...,...,...,...
968,0x25431341a5800759268a6ac1d3cd91c029d7d9caSPLI...,0.299241,0.000000,2.992408e-01
969,0xd6b1fbcbe39e33a3d5d9014b024f511be3564ee5SPLI...,24220.148513,0.000000,2.422015e+04
972,0xbf891e7efcc98a8239385d3172ba10ad593c7886SPLI...,38.478352,0.000000,3.847835e+01
974,0xb14d56ce29dade0316ea5df06d491636a7c4f3c8SPLI...,0.050238,0.050238,4.000002e-12


In [13]:
#df[['First Name', 'Last Name']] = df["Student_details"].apply(lambda x: pd.Series(str(x).split("_")))
userDf[['user', 'pool']] = userDf.Users_Pool.apply(lambda x: pd.Series(str(x).split("SPLIT")))

In [14]:
def totalChainTransfersTo(row):
    return dfChain[dfChain.TO_ADDRESS == row['user']]['N_TRANSACTIONS'].sum()
def totalChainTransfersFrom(row):
    return dfChain[dfChain.FROM_ADDRESS == row['user']]['N_TRANSACTIONS'].sum()

def firstTransfer(row):
    return dfChain[dfChain.FROM_ADDRESS == row['user']]['FIRST_TRANSACTION'].min()
def lastTransfer(row):
    return dfChain[dfChain.FROM_ADDRESS == row['user']]['LAST_TRANSACTION'].max()    

def totalAssetTransfersTo(row):
    return dfAsset[dfAsset.TO_ADDRESS == row['user']]['TOTAL'].sum()
def totalAssetTransfersFrom(row):
    return dfAsset[dfAsset.FROM_ADDRESS == row['user']]['TOTAL'].sum()





In [15]:
for chain in chains : 
    dfChain = df[df.CHAIN == chain]
    #asset = 'USDC'
    userDf['totalChainTransfersTo' + chain] = userDf.apply(totalChainTransfersTo, axis = 1)
    userDf['totalChainTransfersFrom' + chain] = userDf.apply(totalChainTransfersFrom, axis = 1)
    userDf['firstTransfer' + chain] = userDf.apply(firstTransfer, axis = 1)
    userDf['lastTransfer' + chain] = userDf.apply(lastTransfer, axis = 1)
    userDf['firstTransfer' + chain] = (userDf['firstTransfer' + chain] - date).dt.days
    userDf['lastTransfer' + chain] = (userDf['lastTransfer' + chain] - date).dt.days

    for asset in topAssets :
        dfAsset = dfChain[dfChain.SYMBOL == asset]
        userDf['totalTransfersTo' + chain + asset] = userDf.apply(totalAssetTransfersTo, axis = 1)
        userDf['totalChainTransfersTo' + chain + asset] = userDf.apply(totalAssetTransfersFrom, axis = 1)


In [29]:
userDf.columns

Index(['Users_Pool', 'totalMints', 'totalBurns', 'NetMints', 'user', 'pool',
       'totalChainTransfersToOPTI', 'totalChainTransfersFromOPTI',
       'firstTransferOPTI', 'lastTransferOPTI',
       ...
       'totalChainTransfersToPOLYWMATIC', 'totalTransfersToPOLYWBTC',
       'totalChainTransfersToPOLYWBTC', 'totalTransfersToPOLYUNI-V2',
       'totalChainTransfersToPOLYUNI-V2', 'totalTransfersToPOLYMIM',
       'totalChainTransfersToPOLYMIM', 'totalTransfersToPOLYDAI.e',
       'totalChainTransfersToPOLYDAI.e', 'Withdraw'],
      dtype='object', length=143)

In [16]:
veloDfY = veloDf[veloDf.Time >= date]
veloDfY['Users_Pool'] = veloDfY.ORIGIN_FROM_ADDRESS + "SPLIT" + veloDfY.POOL_NAME

/tmp/ipykernel_46420/3477743160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  veloDfY['Users_Pool'] = veloDfY.ORIGIN_FROM_ADDRESS + "SPLIT" + veloDfY.POOL_NAME


In [17]:
veloDfBurned = veloDfY[veloDfY.LP_TOKEN_ACTION == 'burn']

In [18]:

userDf['Withdraw'] = userDf.Users_Pool.isin(veloDfBurned.Users_Pool)

In [21]:
userDf.columns

Index(['Users_Pool', 'totalMints', 'totalBurns', 'NetMints', 'user', 'pool',
       'totalChainTransfersToOPTI', 'totalChainTransfersFromOPTI',
       'firstTransferOPTI', 'lastTransferOPTI',
       ...
       'totalChainTransfersToPOLYWMATIC', 'totalTransfersToPOLYWBTC',
       'totalChainTransfersToPOLYWBTC', 'totalTransfersToPOLYUNI-V2',
       'totalChainTransfersToPOLYUNI-V2', 'totalTransfersToPOLYMIM',
       'totalChainTransfersToPOLYMIM', 'totalTransfersToPOLYDAI.e',
       'totalChainTransfersToPOLYDAI.e', 'Withdraw'],
      dtype='object', length=143)

In [26]:
topAssets

Index(['USDC', 'WETH', 'USDT', 'DAI', 'USDC.e', 'OP', 'sUSD', 'JLP', 'VELO',
       'USDT.e', 'WMATIC', 'WBTC', 'UNI-V2', 'MIM', 'DAI.e'],
      dtype='object')

In [37]:
from fastai.tabular.all import *

# Here we want to predict if a users will withdraw or not in the next X days based on their historical interactions with AAVE 

cat_names = ['pool']
cont_names = ['totalMints', 'totalBurns']
for chain in chains : 
    cont_names.append('totalChainTransfersTo' + chain)
    cont_names.append('totalChainTransfersFrom' + chain)
    cont_names.append('firstTransfer' + chain)
    cont_names.append('lastTransfer' + chain)

    for asset in ['USDC'] :
        x = 1
        #cont_names.append('totalTransfersTo' + chain + asset)
        #cont_names.append('totalChainTransfersTo' + chain + asset)

splits = RandomSplitter(valid_pct=0.25)(range_of(userDf))

procs = [Categorify, FillMissing, Normalize]
dls = TabularDataLoaders.from_df(userDf, '', procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names='Withdraw', splits=splits, bs=32)
learn = tabular_learner(dls, metrics=accuracy)



In [38]:
learn.fit_one_cycle(5)


epoch,train_loss,valid_loss,accuracy,time
0,0.755302,0.672699,0.656410,00:00
1,0.709033,0.626402,0.723077,00:00
2,0.653357,0.559573,0.774359,00:00
3,0.603243,0.539794,0.789744,00:00
4,0.565833,0.539245,0.800000,00:00
